In [1]:
# General imports
import os
import sys
import numpy as np

# Gymnasium
import gymnasium as gym

# Sinergym
import sinergym
from sinergym.utils.rewards import ExpReward
from src.sinergym_wrapper import SinergymWrapper

In [2]:
# Add the EnergyPlus path to the system path
sys.path.append('./EnergyPlus-23-1-0')
# Set the EnergyPlus path as an environment variable
os.environ['EPLUS_PATH'] = './EnergyPlus-23-1-0'

In [3]:
# Environment ID
id = "Eplus-5zone-hot-continuous-stochastic-v1"

# Weather
weather_files = ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']

# mu, sigma and theta for the weather variability
# In the original version, weather_variability was a triple (mu, sigma, theta)
# that only affected the drybulb (outdoor temperature), while now it is a dictionary
# with the mean, sigma and theta for each weather variable we want to vary
# NOTE: Check apply_weather_variability method in CustomModelJSON class
weather_variability = {
    'drybulb': np.array([5.53173187e+00, 0.00000000e+00, 2.55034944e-03]), 
    'relhum': np.array([1.73128872e+01, 0.00000000e+00, 2.31712760e-03]), 
    'winddir': np.array([7.39984654e+01, 0.00000000e+00, 4.02298013e-04]), 
    'dirnorrad': np.array([3.39506556e+02, 0.00000000e+00, 9.78192172e-04]), 
    'windspd': np.array([1.64655725e+00, 0.00000000e+00, 3.45045547e-04])}

# Reward function and kwargs
reward = ExpReward
reward_kwargs = {
    'temperature_variables': ['air_temperature'],
    'energy_variables': ['HVAC_electricity_demand_rate'],
    'range_comfort_winter': [20, 23],
    'range_comfort_summer': [23, 26],
    'energy_weight': 0.1
}

In [4]:
# Create the environment
env = gym.make(
    id=id,
    weather_files=weather_files,
    reward=reward,
    reward_kwargs=reward_kwargs,
    weather_variability=weather_variability
)

# Wrap the environment (to account for the different weather variability)
env =  SinergymWrapper(env)  

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-5zone-hot-continuous-stochastic-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode:

In [5]:
env.unwrapped.info()


    #==================================================================================#
                                ENVIRONMENT NAME: 5zone-hot-continuous-stochastic-v1
    #==================================================================================#
    #----------------------------------------------------------------------------------#
                                ENVIRONMENT INFO:
    #----------------------------------------------------------------------------------#
    - Building file: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
    - Zone names: ['PLENUM-1', 'SPACE1-1', 'SPACE2-1', 'SPACE3-1', 'SPACE4-1', 'SPACE5-1']
    - Weather file(s): ['USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw']
    - Current weather used: /home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/sinergym/data/weather/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw
    - Episodes executed: 

In [6]:
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print("\n\nOBSERVATION: ", obs)
            print("REWARD: ", reward)
            print("TERMINATED: ", terminated)
            print("TRUNCATED: ", truncated)
            print("INFO: ", info)
        break

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-stochastic-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3_Randomized.epw', '-d', '/home/luigi/Do

In [7]:
from stable_baselines3 import PPO

# Create the PPO agent
model = PPO('MlpPolicy', env, verbose=1)

# Train the agent
model.learn(total_timesteps=10000)

# Save the agent
model.save("ppo_agent")

# To load the trained agent
# model = PPO.load("ppo_agent")


2024-02-06 18:24:55.437214: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 18:24:55.467703: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 18:24:55.467729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 18:24:55.468654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 18:24:55.474625: I tensorflow/core/platform/cpu_feature_guar

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-stochastic-v1] [Episode 2]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run2]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1/Eplus-env-sub_run2/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/home/luigi/Documents/SmartEnergyOptimizationRL/Eplus-env-eplus-env-v1-res1

Progress: |***************************************************************************************************| 99%


In [8]:
env.close()

Progress: |***************************************************************************************************| 99%


[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-stochastic-v1]


## MODEL

In [9]:
import torch.nn as nn

# A constant to use for any configuration that should be deprecated
# (to check, whether this config has actually been assigned a proper value or
# not).
DEPRECATED_VALUE = -1

model_config = {
    # Experimental flag.
    # If True, try to use a native (tf.keras.Model or torch.Module) default
    # model instead of our built-in ModelV2 defaults.
    # If False (default), use "classic" ModelV2 default models.
    # Note that this currently only works for:
    # 1) framework != torch AND
    # 2) fully connected and CNN default networks as well as
    # auto-wrapped LSTM- and attention nets.
    "_use_default_native_models": False,
    # Experimental flag.
    # If True, user specified no preprocessor to be created
    # (via config._disable_preprocessor_api=True). If True, observations
    # will arrive in model as they are returned by the env.
    "_disable_preprocessor_api": False,
    # Experimental flag.
    # If True, RLlib will no longer flatten the policy-computed actions into
    # a single tensor (for storage in SampleCollectors/output files/etc..),
    # but leave (possibly nested) actions as-is. Disabling flattening affects:
    # - SampleCollectors: Have to store possibly nested action structs.
    # - Models that have the previous action(s) as part of their input.
    # - Algorithms reading from offline files (incl. action information).
    "_disable_action_flattening": False,

    # === Built-in options ===
    # FullyConnectedNetwork (tf and torch): rllib.models.tf|torch.fcnet.py
    # These are used if no custom model is specified and the input space is 1D.
    # Number of hidden layers to be used.
    "fcnet_hiddens": [256, 256],
    # Activation function descriptor.
    # Supported values are: "tanh", "relu", "swish" (or "silu"),
    # "linear" (or None).
    "fcnet_activation": "tanh",

    # VisionNetwork (tf and torch): rllib.models.tf|torch.visionnet.py
    # These are used if no custom model is specified and the input space is 2D.
    # Filter config: List of [out_channels, kernel, stride] for each filter.
    # Example:
    # Use None for making RLlib try to find a default filter setup given the
    # observation space.
    "conv_filters": None,
    # Activation function descriptor.
    # Supported values are: "tanh", "relu", "swish" (or "silu"),
    # "linear" (or None).
    "conv_activation": "relu",

    # Some default models support a final FC stack of n Dense layers with given
    # activation:
    # - Complex observation spaces: Image components are fed through
    #   VisionNets, flat Boxes are left as-is, Discrete are one-hot'd, then
    #   everything is concated and pushed through this final FC stack.
    # - VisionNets (CNNs), e.g. after the CNN stack, there may be
    #   additional Dense layers.
    # - FullyConnectedNetworks will have this additional FCStack as well
    # (that's why it's empty by default).
    "post_fcnet_hiddens": [],
    "post_fcnet_activation": "relu",

    # For DiagGaussian action distributions, make the second half of the model
    # outputs floating bias variables instead of state-dependent. This only
    # has an effect is using the default fully connected net.
    "free_log_std": False,
    # Whether to skip the final linear layer used to resize the hidden layer
    # outputs to size `num_outputs`. If True, then the last hidden layer
    # should already match num_outputs.
    "no_final_linear": False,
    # Whether layers should be shared for the value function.
    "vf_share_layers": True,

    # == LSTM ==
    # Whether to wrap the model with an LSTM.
    "use_lstm": False,
    # Max seq len for training the LSTM, defaults to 20.
    "max_seq_len": 20,
    # Size of the LSTM cell.
    "lstm_cell_size": 256,
    # Whether to feed a_{t-1} to LSTM (one-hot encoded if discrete).
    "lstm_use_prev_action": False,
    # Whether to feed r_{t-1} to LSTM.
    "lstm_use_prev_reward": False,
    # Whether the LSTM is time-major (TxBx..) or batch-major (BxTx..).
    "_time_major": False,

    # == Attention Nets (experimental: torch-version is untested) ==
    # Whether to use a GTrXL ("Gru transformer XL"; attention net) as the
    # wrapper Model around the default Model.
    "use_attention": False,
    # The number of transformer units within GTrXL.
    # A transformer unit in GTrXL consists of a) MultiHeadAttention module and
    # b) a position-wise MLP.
    "attention_num_transformer_units": 1,
    # The input and output size of each transformer unit.
    "attention_dim": 64,
    # The number of attention heads within the MultiHeadAttention units.
    "attention_num_heads": 1,
    # The dim of a single head (within the MultiHeadAttention units).
    "attention_head_dim": 32,
    # The memory sizes for inference and training.
    "attention_memory_inference": 50,
    "attention_memory_training": 50,
    # The output dim of the position-wise MLP.
    "attention_position_wise_mlp_dim": 32,
    # The initial bias values for the 2 GRU gates within a transformer unit.
    "attention_init_gru_gate_bias": 2.0,
    # Whether to feed a_{t-n:t-1} to GTrXL (one-hot encoded if discrete).
    "attention_use_n_prev_actions": 0,
    # Whether to feed r_{t-n:t-1} to GTrXL.
    "attention_use_n_prev_rewards": 0,

    # == Atari ==
    # Set to True to enable 4x stacking behavior.
    "framestack": True,
    # Final resized frame dimension
    "dim": 84,
    # (deprecated) Converts ATARI frame to 1 Channel Grayscale image
    "grayscale": False,
    # (deprecated) Changes frame to range from [-1, 1] if true
    "zero_mean": True,

    # === Options for custom models ===
    # Name of a custom model to use
    "custom_model": None,
    # Extra options to pass to the custom classes. These will be available to
    # the Model's constructor in the model_config field. Also, they will be
    # attempted to be passed as **kwargs to ModelV2 models. For an example,
    # see rllib/models/[tf|torch]/attention_net.py.
    "custom_model_config": {},
    # Name of a custom action distribution to use.
    "custom_action_dist": None,
    # Custom preprocessors are deprecated. Please use a wrapper class around
    # your environment instead to preprocess observations.
    "custom_preprocessor": None,

    # Deprecated keys:
    # Use `lstm_use_prev_action` or `lstm_use_prev_reward` instead.
    "lstm_use_prev_action_reward": DEPRECATED_VALUE,
    

    "fcnet_activation": lambda: nn.Sequential(nn.Tanh(), nn.Dropout(p=0.1)),
    "num_dropout_evals": 10,
}

In [10]:
from src.uncertain_ppo import UncertainPPO

agent = UncertainPPO(config={'model': model_config})

Could not instantiate TBXLogger: No module named 'tensorboardX'.
2024-02-06 18:25:15,431	INFO worker.py:1538 -- Started a local Ray instance.
2024-02-06 18:25:19,752	ERROR actor_manager.py:486 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=29025, ip=192.168.1.40, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fbef53a4f70>)
  File "/home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 662, in __init__
    self.policy_dict, self.is_policy_to_train = self.config.get_multi_agent_setup(
  File "/home/luigi/Documents/SmartEnergyOptimizationRL/.venv/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm_config.py", line 2182, in get_multi_agent_setup
    raise ValueError(
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no 

ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!